Установим необходимые библиотеки

In [ ]:
!pip install transformers datasets accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

Умпортируем зависимости

In [ ]:
import pandas as pd
import random
import os
import torch
from datasets import load_dataset, Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

Загрузим датасет и разделим его на выборки

In [ ]:
data_path = "./books.txt"

with open(data_path, 'r', encoding='utf-8') as file:
    data = file.readlines()

data = [line.strip() for line in data if line.strip()]

data_size = len(data)
print('Total lines: ', data_size)

val_size = round(data_size / 2)

random.seed(1234)

val_indices = random.sample(range(data_size), val_size)
train_data = [data[i] for i in range(data_size) if i not in val_indices]
val_data = [data[i] for i in val_indices]

with open("train.txt", 'w', encoding='utf-8') as file:
    file.write("\n".join(train_data))

with open("valid.txt", 'w', encoding='utf-8') as file:
    file.write("\n".join(val_data))

Total lines:  865


Создаём и настраиванем токенизатор. Форматируем датасет

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained('gpt2')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

train_texts = open('train.txt').read().splitlines()
val_texts = open('valid.txt').read().splitlines()

train_dataset = Dataset.from_dict({"text": train_texts})
val_dataset = Dataset.from_dict({"text": val_texts})

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

train_dataset = train_dataset.map(lambda examples: {'labels': examples['input_ids']}, batched=True)
val_dataset = val_dataset.map(lambda examples: {'labels': examples['input_ids']}, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/433 [00:00<?, ? examples/s]

Map:   0%|          | 0/432 [00:00<?, ? examples/s]

Map:   0%|          | 0/433 [00:00<?, ? examples/s]

Map:   0%|          | 0/432 [00:00<?, ? examples/s]

Настраиваем тренер

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Запускаем тренировку модели и сохраняем её

In [7]:
trainer.train()

model.save_pretrained('./gpt2_model')
tokenizer.save_pretrained('./gpt2_model')

Epoch,Training Loss,Validation Loss
1,No log,0.661564


Epoch,Training Loss,Validation Loss
1,No log,0.661564
2,No log,0.640784
3,No log,0.634226


('./gpt2_model/tokenizer_config.json',
 './gpt2_model/special_tokens_map.json',
 './gpt2_model/vocab.json',
 './gpt2_model/merges.txt',
 './gpt2_model/added_tokens.json')

Загружаем токенизатор и модель. Генерируем текст

In [12]:
model_path = "./gpt2_model"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.to("cpu")

def generate_text(prompt, max_length=100, temperature=0.7, top_k=50, top_p=0.5):
    inputs = tokenizer.encode(prompt, return_tensors="pt").to("cpu")
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "Привет"
print(generate_text(prompt))

Привет в призвет в призвет в призвет в призвет в призвет в призвет в призвет в призвет в пр
